In [99]:
from math import sqrt
import numpy as np

class knn:
    # Calculate the Euclidean distance between two vectors
    def euclidean_distance(self, row1, row2):
        distance = 0.0
        subtract = np.subtract(row1, row2)
        for s in subtract:
            distance += s*s
        
        return sqrt(distance)

    # Locate the most similar neighbors
    def get_neighbors(self, train, test_row, num_neighbors):
        distances = list()
        for train_row in train:
            dist = self.euclidean_distance(test_row, train_row)
            distances.append((train_row, dist))
        distances.sort(key=lambda tup: tup[1])
        neighbors = list()
        for i in range(num_neighbors):
            neighbors.append(distances[i][0])
        
        return neighbors

    # Make a prediction with neighbors
    def predict_classification(self, train, test_row, num_neighbors):
        neighbors = self.get_neighbors(train, test_row, num_neighbors)
        output_values = [row[-1] for row in neighbors]
        prediction = max(set(output_values), key=output_values.count)
    
        return prediction
        

Split dataset into a train, test and validation

In [100]:
from sklearn import datasets
from sklearn.model_selection import train_test_split

mnist = datasets.load_digits()

X, y = mnist["data"], mnist["target"]
X, X_test, y, y_test = train_test_split(X, y, stratify=y, test_size=0.2)
X_train, X_val, y_train, y_val = train_test_split(X, y, stratify=y, test_size=0.2)

Predicted: 6.0
6


After trying different "k"s and distance functions I found that k= and d= works best.

In [ ]:
# define model parameter
num_neighbors = 5
# predict the label
knn = knn()
y_train = y_train.reshape(-1, 1)

label = knn.predict_classification(np.append(X_train, y_train, axis=1), np.append(X_val[0], y_val[0]), num_neighbors)
print('Predicted: %s' % (label))
print(y_val[0])


Report Train, validation and test error

In [ ]:
from sklearn.metrics import mean_squared_error

mean_squared_error(y_true, y_pred)





